In [1]:
# !pip install selenium

In [2]:
import pickle
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import requests
from time import sleep

# Progress Bar
from tqdm import tqdm

In [3]:
# Path to the Chrome WebDriver executable
driver = webdriver.Chrome()

In [4]:
def extract_journal(doi):
    # just get text after 'j.' until another '.'
    match = re.search(r'j\.([^\.]+)', doi)
    if match:
        name = match.group(1)
        return name

In [5]:
data = pd.read_csv(os.getcwd() + '/data/body_extracted.csv', index_col=0)

dois = data['doi']

data['doi'] = data['doi'].apply(extract_journal)
threshold = 22/len(data) # the remaining journals need to at least contain 22 articles
counts = data['doi'].value_counts(normalize=True)
data = data.loc[data['doi'].isin(counts[counts > threshold].index), :]
journals = data['doi'].unique()

In [29]:
def scape_name(url):
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source)
    publication_elem = soup.find('div', class_='Publication')

    if publication_elem is None:
        return ''
    
    title_elem = publication_elem.find('h2', class_="publication-title")
    if title_elem is None:
        title_elem = publication_elem.find('a', class_="publication-brand-title-link")
    elem = title_elem.find('span', class_="anchor-text")
    if elem is not None:
        return elem.text
    else:
        print('bad', url)
        return ''


In [30]:
# get all journal
journal_map = {}
for journal in tqdm(journals):
    for doi in dois:
        match = re.search(r'j\.([^\.]+)', doi)
        if match:
            name = match.group(1)
            if name == journal:
                url = 'https://doi.org/' + doi
                journal_name = scape_name(url)
                journal_map[journal] = journal_name
#                 print(doi, journal_name)
                break

# Add missing journal (this one redirect to different url)
journal_map['amsu'] = 'Annals of Medicine and Surgery' 

100%|██████████| 378/378 [04:15<00:00,  1.48it/s]


In [26]:
file_path = 'data/journal_abv_dictionary.pkl'

# Open the file in binary write mode and save the data
with open(file_path, 'wb') as file:
    pickle.dump(journal_map, file)

In [31]:
file_path = 'data/journal_abv_dictionary.pkl'
with open(file_path, 'rb') as f:
    print(pickle.load(f))


{'aap': 'Accident Analysis & Prevention', 'actatropica': 'Acta Tropica', 'ajhg': 'The American Journal of Human Genetics', 'aca': 'Analytica Chimica Acta', 'anbehav': 'Animal Behaviour', 'apergo': 'Applied Ergonomics', 'brat': 'Behaviour Research and Therapy', 'bbrc': 'Biochemical and Biophysical Research Communications', 'bcp': 'Biochemical Pharmacology', 'biocon': 'Biological Conservation', 'biopsych': 'Biological Psychiatry', 'brainres': 'Brain Research', 'carres': 'Carbohydrate Research', 'ces': 'Chemical Engineering Science', 'chemgeo': 'Chemical Geology', 'cognition': 'Cognition', 'combustflame': 'Combustion and Flame', 'corsci': 'Corrosion Science', 'cortex': 'Cortex', 'ydbio': 'Developmental Biology', 'epsl': 'Earth and Planetary Science Letters', 'earscirev': 'Earth-Science Reviews', 'electacta': 'Electrochimica Acta', 'envres': 'Environmental Research', 'ejphar': 'European Journal of Pharmacology', 'eurpolymj': 'European Polymer Journal', 'exer': 'Experimental Eye Research', 